In [1]:
import numpy as np
import pandas as pd
import vitaldb as vf
import matplotlib.pyplot as plt

In [2]:
################################### UTILITIES ##########################################

def select_cases(include_tracks, exclude_tracks):
    """extracts the relevant columns from the cases.csv file and applies one-hot encoding to text columns
    Returns:
        pandas.df
    """
    df = pd.read_csv('data/info_vitaldb/cases.csv')

    # Select relevant columns to be used
    relevant_columns = ['caseid','age','sex','height','weight','ane_type']
    df = df[relevant_columns]

    ### Encode Gender
    df = pd.get_dummies(df, columns=['sex'], drop_first=True)

    ### Collect only general anesthesia without spinal and sedationalgesia
    clinical_info = df[df['ane_type'] == 'General'].drop(columns=['ane_type'])

    ### Extract the caseids of the cases that have all the required tracks listed in the track_names variable in relevant_tracks.py
    df = pd.read_csv('data/info_vitaldb/tracks_sorted.csv')

    include = df[include_tracks].all(1)
    exclude = ~df[exclude_tracks].any(axis='columns')

    filtered_tracks =  list(df['caseid'].loc[include*exclude])

    ### Filter the clinical info based on the caseids
    clinical_info = clinical_info[clinical_info['caseid'].isin(filtered_tracks)]
                                  
    selected_cases = clinical_info.reset_index(drop=True)

    return selected_cases

In [ ]:
include_track_names = [
'SNUADC/ART',               # Arterial pressure wave                                W/500   mmHg
#'SNUADC/CVP',               # Central venous pressure wave                          W/500   mmHg
'SNUADC/ECG_II',            # ECG lead II wave                                      W/500   mV
#'SNUADC/ECG_V5',            # ECG lead V5 wave                                      W/500   mV
#'SNUADC/FEM',               # Femoral arterial pressure wave                        W/500   mmHg
#'SNUADC/PLETH',             # Plethysmography wave                                  W/500   unitless
'Solar8000/ART_DBP',        # Diastolic arterial pressure                           N       mmHg
'Solar8000/ART_MBP',        # Mean arterial pressure                                N       mmHg
'Solar8000/ART_SBP',        # Systolic arterial pressure                            N       mmHg
#'Solar8000/BT',             # Body temperature                                      N       °C
#'Solar8000/CVP',            # Central venous pressure                               N       mmHg
'Solar8000/ETCO2',          # End-tidal CO2                                         N       mmHg
#'Solar8000/FEM_DBP',        # Femoral diastolic arterial pressure                   N       mmHg
#'Solar8000/FEM_MBP',        # Femoral mean arterial pressure                        N       mmHg
#'Solar8000/FEM_SBP',        # Femoral systolic arterial pressure                    N       mmHg
'Solar8000/FEO2',           # Fraction of expired O2                                N       %
'Solar8000/FIO2',           # Fraction of inspired O2                               N       %
#'Solar8000/GAS2_EXPIRED',   # Expiratory volatile concentration                     N       %
#'Solar8000/GAS2_INSPIRED',  # Inspiratory volatile concentration                    N       %
'Solar8000/HR',             # Heart rate                                            N       /min
'Solar8000/INCO2',          # Inspiratory CO2                                       N       mmHg
'Solar8000/NIBP_DBP',       # Non-invasive diastolic arterial pressure              N       mmHg
'Solar8000/NIBP_MBP',       # Non-invasive mean arterial pressure                   N       mmHg
'Solar8000/NIBP_SBP',       # Non-invasive systolic arterial pressure               N       mmHg
#'Solar8000/PA_DBP',         # Pulmonary diastolic arterial pressure                 N       mmHg
#'Solar8000/PA_MBP',         # Pulmonary mean arterial pressure                      N       mmHg
#'Solar8000/PA_SBP',         # Pulmonary systolic arterial pressure                  N       mmHg
#'Solar8000/PLETH_HR',       # Heart rate based on plethysmography                   N       /min
'Solar8000/PLETH_SPO2',     # Percutaneous oxygen saturation                        N       %
#'Solar8000/RR',             # Respiratory rate based on ECG                         N       /min
'Solar8000/RR_CO2',         # Respiratory rate based on capnography                 N       /min
#'Solar8000/ST_AVF',         # ST segment in lead aVF                                N       mm
#'Solar8000/ST_AVL',         # ST segment in lead aVL                                N       mm
#'Solar8000/ST_AVR',         # ST segment in lead aVR                                N       mm
#'Solar8000/ST_I',           # ST segment in lead I                                  N       mm
#'Solar8000/ST_II',          # ST segment in lead II                                 N       mm
#'Solar8000/ST_III',         # ST segment in lead III                                N       mm
#'Solar8000/ST_V5'           # ST segment in lead V5                                 N       mm
#'Solar8000/VENT_COMPL',     # Airway compliance (from ventilator)                   N       mL/mbar
#'Solar8000/VENT_INSP_TM',   # Inspiratory time (from ventilator)                    N       sec
#'Solar8000/VENT_MAWP',      # Mean airway pressure (from ventilator)                N       mbar
#'Solar8000/VENT_MEAS_PEEP', # Positive end-expiratory pressure (from ventilator)    N       mbar
#'Solar8000/VENT_MV',        # Minute ventilation (from ventilator)                  N       L/min
#'Solar8000/VENT_PIP',       # Peak inspiratory pressure (from ventilator)           N       mbar
#'Solar8000/VENT_PPLAT',     # Plateau pressure (from ventilator)                    N       mbar
#'Solar8000/VENT_RR',        # Respiratory rate (from ventilator)                    N       /min
#'Solar8000/VENT_SET_FIO2',  # Set fraction of inspired O2 (from ventilator)         N       %
#'Solar8000/VENT_SET_PCP',   # Set peak inspiratory pressure                         N       cmH2O
#'Solar8000/VENT_SET_TV',    # Set tidal volume in volume control mode               N       mL
#'Solar8000/VENT_TV',        # Measured tidal volume (from ventilator)               N       mL
#'Primus/AWP',               # Airway pressure wave                                  W/62.5  hPa
'Primus/CO2',               # Capnography wave                                      W/62.5  mmHg
#'Primus/COMPLIANCE',        # Airway compliance                                     N       mL/mbar
'Primus/ETCO2',             # End-tidal CO2                                         N       mmHg
#'Primus/EXP_DES',           # Expiratory desflurane pressure                        N       kPa
#'Primus/EXP_SEVO',          # Expiratory sevoflurane pressure                       N       kPa
#'Primus/FEN2O',             # Fraction of expired N2O                               N       %
'Primus/FEO2',              # Fraction of expired O2                                N       %
#'Primus/FIN2O',             # Fraction of inspired N2O                              N       %
'Primus/FIO2',              # Fraction of inspired O2                               N       %
#'Primus/FLOW_AIR',          # Flow rate of air                                      N       mL/min
#'Primus/FLOW_N2O',          # Flow rate of N2O                                      N       mL/min
#'Primus/FLOW_O2',           # Flow rate of O2                                       N       mL/min
'Primus/INCO2',             # Inspiratory CO2                                       N       mmHg
#'Primus/INSP_DES',          # Inspiratory desflurane pressure                       N       kPa
#'Primus/INSP_SEVO',         # Inspiratory sevoflurane pressure                      N       kPa
#'Primus/MAC',               # Minimum alveolar concentration of volatile            N       nitless
#'Primus/MAWP_MBAR',         # Mean airway pressure                                  N       mbar
'Primus/MV',                # Minute volume                                         N       L
#'Primus/PEEP_MBAR',         # Positive end expiratory pressure (PEEP)               N       mbar
#'Primus/PIP_MBAR',          # Peak inspiratory pressure                             N       mbar
#'Primus/PPLAT_MBAR',        # Plateau pressure                                      N       mbar
#'Primus/RR_CO2',            # Respiratory rate based on capnography                 N       /min
'Primus/TV',                # Tidal volume                                          N       mL
#'Primus/VENT_LEAK',         # Ventilator leakage                                    N       mL/min
'Orchestra/PPF20_CE',       # Effect-site concentration (propofol 20 mg/mL)         N       mcg/mL
'Orchestra/PPF20_CP',       # Plasma concentration (propofol 20 mg/mL)              N       mcg/mL
'Orchestra/PPF20_CT',       # Target concentration (propofol 20 mg/mL)              N       mcg/mL
'Orchestra/PPF20_RATE',     # Infusion rate (propofol 20 mg/mL)                     N       mL/hr
'Orchestra/PPF20_VOL',      # Infused volume (propofol 20 mg/mL)                    N       mL
'Orchestra/RFTN20_CE',      # Effect-site concentration (remifentanil 20 mcg/mL)    N       ng/mL
'Orchestra/RFTN20_CP',      # Plasma concentration (remifentanil 20 mcg/mL)         N       ng/mL
'Orchestra/RFTN20_CT',      # Target concentration (remifentanil 20 mcg/mL)         N       ng/mL
'Orchestra/RFTN20_RATE',    # Infusion rate (remifentanil 20 mcg/mL)                N       mL/hr
'Orchestra/RFTN20_VOL',     # Infused volume (remifentanil 20 mcg/mL)               N       mL
'BIS/BIS',                  # Bispectral index value                                N       unitless
#'BIS/EEG1_WAV',             # EEG wave from channel 1                               W/128   uV
#'BIS/EEG2_WAV',             # EEG wave from channel 2                               W/128   uV
]

exclude_track_names = [
'Primus/EXP_DES',           # Expiratory desflurane pressure                        N       kPa
'Primus/EXP_SEVO',          # Expiratory sevoflurane pressure                       N       kPa
#'Orchestra/PPF20_CE',       # Effect-site concentration (propofol 20 mg/mL)         N       mcg/mL
#'Orchestra/PPF20_CP',       # Plasma concentration (propofol 20 mg/mL)              N       mcg/mL
#'Orchestra/PPF20_CT',       # Target concentration (propofol 20 mg/mL)              N       mcg/mL
#'Orchestra/PPF20_RATE',     # Infusion rate (propofol 20 mg/mL)                     N       mL/hr
#'Orchestra/PPF20_VOL',      # Infused volume (propofol 20 mg/mL)                    N       mL
#'Orchestra/RFTN20_CE',      # Effect-site concentration (remifentanil 20 mcg/mL)    N       ng/mL
#'Orchestra/RFTN20_CP',      # Plasma concentration (remifentanil 20 mcg/mL)         N       ng/mL
#'Orchestra/RFTN20_CT',      # Target concentration (remifentanil 20 mcg/mL)         N       ng/mL
#'Orchestra/RFTN20_RATE',    # Infusion rate (remifentanil 20 mcg/mL)                N       mL/hr
#'Orchestra/RFTN20_VOL',     # Infused volume (remifentanil 20 mcg/mL)               N       mL
]

In [ ]:
select_cases(include_track_names, exclude_track_names)

In [ ]:
########################################## GENERATE DATASET ##########################################

# This script is used to generate a dataset with only continuous BIS data from the VitalDB
# The data is generated in 1s intervals and saved as vital files in the folder 'data/vitaldb_tiva/'

#################################### TOTAL INTRAVENOUS ANAESTHESIA ###################################

### Select Cases
cases = select_cases(include_track_names,exclude_track_names)
#mod.dataset_info(cases)

### Caseselection with only continuous BIS data
case_list = cases['caseid'].values
#case_list = [13,14]

for n,i in enumerate(case_list):
    #if i < 4640: continue
    samples = vf.VitalFile(ipath = int(i), track_names = ['BIS/BIS'])
    samples = samples.to_numpy(track_names=['BIS/BIS'],interval=1,return_timestamp=True)

    # Filter the zero BIS values at the beginning and end of the recording
    filter = samples[~np.isnan(samples)[:,1],:]
    nonzeroArray = np.nonzero(filter[:,1])

    print(" Fortschritt: %.1f" % (100 * (n+1) / len(case_list)),' % ', end='\r')

    try: 
        filter = filter[nonzeroArray[0][0]:nonzeroArray[0][-1],:]

        ### allow 5% of the BIS data to be NaN
        if np.isnan(filter).sum() / filter.shape[0] < 0.05:
            
            ### Get the relevant track records
            samples = vf.VitalFile(ipath = int(i), track_names = include_track_names).crop(dtfrom=filter[0,0], dtend=filter[-1,0])
            
            ### filter too short recordings (shorter than 1,5 hours)
            if samples.to_numpy(track_names=['BIS/BIS'],interval=1).shape[0] < (3600 * 1.5):
                continue

            ### check if the BIS at the beginning is above 80
            if samples.to_numpy(track_names=['BIS/BIS'],interval=1)[0][0] < 80:
                continue

            ### check, if the infused propofol volume at the start of the recording is below 0.5 mg
            if samples.to_numpy(track_names=['Orchestra/PPF20_VOL'],interval=1)[0][0] > 0.5:
                continue

            ### save the vital file
            samples.to_vital('data/vitaldb_tiva/'+str(i).zfill(4)+'.vital')

    except:
        print('Error in case: ',i)
        continue

In [3]:
include_track_names = [
#'SNUADC/ART',               # Arterial pressure wave                                W/500   mmHg
#'SNUADC/CVP',               # Central venous pressure wave                          W/500   mmHg
#'SNUADC/ECG_II',            # ECG lead II wave                                      W/500   mV
#'SNUADC/ECG_V5',            # ECG lead V5 wave                                      W/500   mV
#'SNUADC/FEM',               # Femoral arterial pressure wave                        W/500   mmHg
#'SNUADC/PLETH',             # Plethysmography wave                                  W/500   unitless
'Solar8000/ART_DBP',        # Diastolic arterial pressure                           N       mmHg
'Solar8000/ART_MBP',        # Mean arterial pressure                                N       mmHg
'Solar8000/ART_SBP',        # Systolic arterial pressure                            N       mmHg
#'Solar8000/BT',             # Body temperature                                      N       °C
#'Solar8000/CVP',            # Central venous pressure                               N       mmHg
'Solar8000/ETCO2',          # End-tidal CO2                                         N       mmHg
#'Solar8000/FEM_DBP',        # Femoral diastolic arterial pressure                   N       mmHg
#'Solar8000/FEM_MBP',        # Femoral mean arterial pressure                        N       mmHg
#'Solar8000/FEM_SBP',        # Femoral systolic arterial pressure                    N       mmHg
'Solar8000/FEO2',           # Fraction of expired O2                                N       %
'Solar8000/FIO2',           # Fraction of inspired O2                               N       %
#'Solar8000/GAS2_EXPIRED',   # Expiratory volatile concentration                     N       %
#'Solar8000/GAS2_INSPIRED',  # Inspiratory volatile concentration                    N       %
'Solar8000/HR',             # Heart rate                                            N       /min
'Solar8000/INCO2',          # Inspiratory CO2                                       N       mmHg
'Solar8000/NIBP_DBP',       # Non-invasive diastolic arterial pressure              N       mmHg
'Solar8000/NIBP_MBP',       # Non-invasive mean arterial pressure                   N       mmHg
'Solar8000/NIBP_SBP',       # Non-invasive systolic arterial pressure               N       mmHg
#'Solar8000/PA_DBP',         # Pulmonary diastolic arterial pressure                 N       mmHg
#'Solar8000/PA_MBP',         # Pulmonary mean arterial pressure                      N       mmHg
#'Solar8000/PA_SBP',         # Pulmonary systolic arterial pressure                  N       mmHg
#'Solar8000/PLETH_HR',       # Heart rate based on plethysmography                   N       /min
'Solar8000/PLETH_SPO2',     # Percutaneous oxygen saturation                        N       %
#'Solar8000/RR',             # Respiratory rate based on ECG                         N       /min
'Solar8000/RR_CO2',         # Respiratory rate based on capnography                 N       /min
#'Solar8000/ST_AVF',         # ST segment in lead aVF                                N       mm
#'Solar8000/ST_AVL',         # ST segment in lead aVL                                N       mm
#'Solar8000/ST_AVR',         # ST segment in lead aVR                                N       mm
#'Solar8000/ST_I',           # ST segment in lead I                                  N       mm
#'Solar8000/ST_II',          # ST segment in lead II                                 N       mm
#'Solar8000/ST_III',         # ST segment in lead III                                N       mm
#'Solar8000/ST_V5'           # ST segment in lead V5                                 N       mm
#'Solar8000/VENT_COMPL',     # Airway compliance (from ventilator)                   N       mL/mbar
#'Solar8000/VENT_INSP_TM',   # Inspiratory time (from ventilator)                    N       sec
#'Solar8000/VENT_MAWP',      # Mean airway pressure (from ventilator)                N       mbar
#'Solar8000/VENT_MEAS_PEEP', # Positive end-expiratory pressure (from ventilator)    N       mbar
#'Solar8000/VENT_MV',        # Minute ventilation (from ventilator)                  N       L/min
#'Solar8000/VENT_PIP',       # Peak inspiratory pressure (from ventilator)           N       mbar
#'Solar8000/VENT_PPLAT',     # Plateau pressure (from ventilator)                    N       mbar
#'Solar8000/VENT_RR',        # Respiratory rate (from ventilator)                    N       /min
#'Solar8000/VENT_SET_FIO2',  # Set fraction of inspired O2 (from ventilator)         N       %
#'Solar8000/VENT_SET_PCP',   # Set peak inspiratory pressure                         N       cmH2O
#'Solar8000/VENT_SET_TV',    # Set tidal volume in volume control mode               N       mL
#'Solar8000/VENT_TV',        # Measured tidal volume (from ventilator)               N       mL
#'Primus/AWP',               # Airway pressure wave                                  W/62.5  hPa
'Primus/CO2',               # Capnography wave                                      W/62.5  mmHg
#'Primus/COMPLIANCE',        # Airway compliance                                     N       mL/mbar
'Primus/ETCO2',             # End-tidal CO2                                         N       mmHg
#'Primus/EXP_DES',           # Expiratory desflurane pressure                        N       kPa
'Primus/EXP_SEVO',          # Expiratory sevoflurane pressure                       N       kPa
#'Primus/FEN2O',             # Fraction of expired N2O                               N       %
'Primus/FEO2',              # Fraction of expired O2                                N       %
#'Primus/FIN2O',             # Fraction of inspired N2O                              N       %
'Primus/FIO2',              # Fraction of inspired O2                               N       %
#'Primus/FLOW_AIR',          # Flow rate of air                                      N       mL/min
#'Primus/FLOW_N2O',          # Flow rate of N2O                                      N       mL/min
#'Primus/FLOW_O2',           # Flow rate of O2                                       N       mL/min
'Primus/INCO2',             # Inspiratory CO2                                       N       mmHg
#'Primus/INSP_DES',          # Inspiratory desflurane pressure                       N       kPa
'Primus/INSP_SEVO',         # Inspiratory sevoflurane pressure                      N       kPa
'Primus/MAC',               # Minimum alveolar concentration of volatile            N       nitless
'Primus/MAWP_MBAR',         # Mean airway pressure                                  N       mbar
'Primus/MV',                # Minute volume                                         N       L
#'Primus/PEEP_MBAR',         # Positive end expiratory pressure (PEEP)               N       mbar
#'Primus/PIP_MBAR',          # Peak inspiratory pressure                             N       mbar
#'Primus/PPLAT_MBAR',        # Plateau pressure                                      N       mbar
#'Primus/RR_CO2',            # Respiratory rate based on capnography                 N       /min
'Primus/TV',                # Tidal volume                                          N       mL
#'Primus/VENT_LEAK',         # Ventilator leakage                                    N       mL/min
#'Orchestra/PPF20_CE',       # Effect-site concentration (propofol 20 mg/mL)         N       mcg/mL
#'Orchestra/PPF20_CP',       # Plasma concentration (propofol 20 mg/mL)              N       mcg/mL
#'Orchestra/PPF20_CT',       # Target concentration (propofol 20 mg/mL)              N       mcg/mL
#'Orchestra/PPF20_RATE',     # Infusion rate (propofol 20 mg/mL)                     N       mL/hr
#'Orchestra/PPF20_VOL',      # Infused volume (propofol 20 mg/mL)                    N       mL
'Orchestra/RFTN20_CE',      # Effect-site concentration (remifentanil 20 mcg/mL)    N       ng/mL
'Orchestra/RFTN20_CP',      # Plasma concentration (remifentanil 20 mcg/mL)         N       ng/mL
'Orchestra/RFTN20_CT',      # Target concentration (remifentanil 20 mcg/mL)         N       ng/mL
'Orchestra/RFTN20_RATE',    # Infusion rate (remifentanil 20 mcg/mL)                N       mL/hr
'Orchestra/RFTN20_VOL',     # Infused volume (remifentanil 20 mcg/mL)               N       mL
'BIS/BIS',                  # Bispectral index value                                N       unitless
#'BIS/EEG1_WAV',             # EEG wave from channel 1                               W/128   uV
#'BIS/EEG2_WAV',             # EEG wave from channel 2                               W/128   uV
]

exclude_track_names = [
#'Primus/EXP_DES',           # Expiratory desflurane pressure                        N       kPa
#'Primus/EXP_SEVO',          # Expiratory sevoflurane pressure                       N       kPa
'Orchestra/PPF20_CE',       # Effect-site concentration (propofol 20 mg/mL)         N       mcg/mL
'Orchestra/PPF20_CP',       # Plasma concentration (propofol 20 mg/mL)              N       mcg/mL
'Orchestra/PPF20_CT',       # Target concentration (propofol 20 mg/mL)              N       mcg/mL
'Orchestra/PPF20_RATE',     # Infusion rate (propofol 20 mg/mL)                     N       mL/hr
'Orchestra/PPF20_VOL',      # Infused volume (propofol 20 mg/mL)                    N       mL
#'Orchestra/RFTN20_CE',      # Effect-site concentration (remifentanil 20 mcg/mL)    N       ng/mL
#'Orchestra/RFTN20_CP',      # Plasma concentration (remifentanil 20 mcg/mL)         N       ng/mL
#'Orchestra/RFTN20_CT',      # Target concentration (remifentanil 20 mcg/mL)         N       ng/mL
#'Orchestra/RFTN20_RATE',    # Infusion rate (remifentanil 20 mcg/mL)                N       mL/hr
#'Orchestra/RFTN20_VOL',     # Infused volume (remifentanil 20 mcg/mL)               N       mL
]

In [4]:
select_cases(include_track_names, exclude_track_names)

,caseid,age,height,weight,sex_M
0,10,72.0,162.5,62.75,True
1,24,57.0,161.8,65.00,True
2,25,66.0,169.7,80.20,True
3,27,50.0,143.2,43.35,False
4,56,23.0,157.9,50.30,False
...,...,...,...,...,...
711,6361,68.0,163.6,55.70,True
712,6362,81.0,163.4,50.30,True
713,6378,68.0,160.7,86.60,False
714,6386,61.0,151.7,43.25,False


In [5]:
########################################## GENERATE DATASET ##########################################

# This script is used to generate a dataset with only continuous BIS data from the VitalDB
# The data is generated in 1s intervals and saved as vital files in the folder 'data/tiva_selection/'

#################################### TOTAL INTRAVENOUS ANAESTHESIA ###################################

### Select Cases
cases = select_cases(include_track_names,exclude_track_names)
#mod.dataset_info(cases)

### Caseselection with only continuous BIS data
case_list = cases['caseid'].values
#case_list = [13,14]

for n,i in enumerate(case_list):
   #if i < 6050: continue
    samples = vf.VitalFile(ipath = int(i), track_names = ['BIS/BIS'])
    samples = samples.to_numpy(track_names=['BIS/BIS'],interval=1,return_timestamp=True)

    # Filter the zero BIS values at the beginning and end of the recording
    filter = samples[~np.isnan(samples)[:,1],:]
    nonzeroArray = np.nonzero(filter[:,1])

    print(" Fortschritt: %.1f" % (100 * (n+1) / len(case_list)),' % ', end='\r')

    try: 
        filter = filter[nonzeroArray[0][0]:nonzeroArray[0][-1],:]

        ### allow 5% of the BIS data to be NaN
        if np.isnan(filter).sum() / filter.shape[0] < 0.05:
            
            ### Get the relevant track records
            samples = vf.VitalFile(ipath = int(i), track_names = include_track_names).crop(dtfrom=filter[0,0], dtend=filter[-1,0])
            
            ### filter too short recordings (shorter than 1,5 hours)
            if samples.to_numpy(track_names=['BIS/BIS'],interval=1).shape[0] < (3600 * 1.5):
                continue

            ### check if the BIS at the beginning is above 80
            if samples.to_numpy(track_names=['BIS/BIS'],interval=1)[0][0] < 80:
                continue

            ### save the vital file
            samples.to_vital('data/vitaldb_sevofl/'+str(i).zfill(4)+'.vital')

    except:
        print('Error in case: ',i)
        continue